<a href="https://colab.research.google.com/github/joheras/MusicaCatedralStoDomingoIER/blob/main/Notebooks/SacarM%C3%BAsica/Dataset_Gigante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comprobando un dataset gigante de música anotada
- https://repository.cloudlab.zhaw.ch/artifactory/deepscores/archives/2017/
- https://tuggeluk.github.io/class_names/class_names.csv
- https://tuggeluk.github.io/class_names/class_names_extended.csv

## Librerías y dataset

In [8]:
import xml.etree.ElementTree as ET

In [2]:
%%capture
!pip install icevision[all]==0.5.1

In [3]:
from icevision.all import *
import cv2

In [4]:
%%capture
!wget https://repository.cloudlab.zhaw.ch/artifactory/deepscores/archives/2017/DeepScores2017_archive0.zip
!unzip DeepScores2017_archive0.zip

## XML $\rightarrow$ JSON

In [9]:
import json

In [14]:
def xml2json(path):
  tree = ET.parse(path)
  root = tree.getroot()
  json = []
  o = Path(path)
  o=o.stem+'.xml'
  o = '/content/images_png/'+o[:o.find('.xml')]+'.png'
  img = cv2.imread(o)
  for object in root.iter("object"):
      label = object.find("name").text
      xml_bbox = object.find("bndbox")
      xmin = int(float(xml_bbox.find("xmin").text)*img.shape[1])
      ymin = int(float(xml_bbox.find("ymin").text)*img.shape[0])
      xmax = int(float(xml_bbox.find("xmax").text)*img.shape[1])
      ymax = int(float(xml_bbox.find("ymax").text)*img.shape[0])
      json.append({"box": [xmin, ymin, xmax, ymax],"label": label})
  return json

In [ ]:
path=Path('/content/xml_annotations')
print(len(path.ls()))
k = 0
for p in path.ls():
  if k%100 == 0: print(k)
  k = k + 1
  j = xml2json(str(p))
  o = p.stem+'.xml'
  o = '/content/json_annotations/'+o[:o.find('.xml')]+'.json'  
  with open(o,'w') as f:
    json.dump(j,f)

## Parser

In [149]:
import pandas as pd
import numpy as np

In [177]:
df = pd.read_csv('/content/class_names_extended.csv',names = [0,1])

In [178]:
class_map = ClassMap(list(np.array(df[1])))

In [179]:
class AnotacionParser(parsers.Parser, parsers.FilepathMixin, parsers.LabelsMixin, parsers.BBoxesMixin):
    pass

In [180]:
AnotacionParser.generate_template()

def __iter__(self) -> Any:
def imageid(self, o) -> Hashable:
def image_width_height(self, o) -> Tuple[int, int]:
    return get_image_size(self.filepath(o))
def filepath(self, o) -> Union[str, Path]:
def bboxes(self, o) -> List[BBox]:
def labels(self, o) -> List[int]:


In [181]:
class AnotacionParser(parsers.Parser, parsers.FilepathMixin, parsers.LabelsMixin, parsers.BBoxesMixin):
    def __init__(self, path_img,path_anotaciones):
        self.path_img = path_img
        self.path_anotaciones= path_anotaciones

    def __iter__(self):
      with os.scandir(self.path_anotaciones) as ficheros:
        for fichero in ficheros:
          if fichero.name!='.ipynb_checkpoints':
            yield fichero.name

    def imageid(self, o) -> Hashable: #o --> nombre de la anotación
      return o[:o.find('.')]

    def filepath(self, o) -> Union[str, Path]:
      path=Path(f"{o[:o.find('.')]}.png")
      return self.path_img / path

    def image_width_height(self, o) -> Tuple[int, int]:
      return get_image_size(self.path_img+'/'+f"{o[:o.find('.')]}.png")

    def labels(self, o) -> List[int]:
      d={df[0][i]:df[1][i] for i in range(0,len(df))}
      path=self.path_anotaciones+'/'+o
      anotacion=json.loads(open(path).read())
      label=[]
      for L in anotacion:
        label.append(d[L['label']])
      return label

    def bboxes(self, o) -> List[BBox]:
      path=self.path_anotaciones+'/'+o
      anotacion=json.loads(open(path).read())
      bbox=[]
      for L in anotacion:
        bbox.append(BBox.from_xyxy(L['box'][0],L['box'][1],L['box'][2],L['box'][3]))
      return bbox

In [182]:
anot=AnotacionParser('/content/images_png','/content/json_annotations')

In [183]:
train_rs, valid_rs = anot.parse()

0it [00:00, ?it/s]

KeyError: ignored